# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235537864711                   -0.50    7.0
  2   -7.250436869041       -1.83       -1.41    1.0
  3   -7.251300696017       -3.06       -2.15    3.0
  4   -7.251225566740   +   -4.12       -2.12    3.0
  5   -7.251333076826       -3.97       -2.77    2.0
  6   -7.251338138310       -5.30       -3.18    2.0
  7   -7.251338783411       -6.19       -4.04    1.0
  8   -7.251338796286       -7.89       -4.36    4.0
  9   -7.251338798592       -8.64       -5.05    2.0
 10   -7.251338798694       -9.99       -5.57    3.0
 11   -7.251338798702      -11.07       -5.86    4.0
 12   -7.251338798704      -11.70       -6.27    2.0
 13   -7.251338798705      -12.49       -6.91    3.0
 14   -7.251338798705      -13.85       -7.29    2.0
 15   -7.251338798705      -14.75       -7.67    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0474662890419271
[ Info: Arnoldi iteration step 2: normres = 0.44268229712448653
[ Info: Arnoldi iteration step 3: normres = 0.8059301862146155
[ Info: Arnoldi iteration step 4: normres = 0.4257097464568376
[ Info: Arnoldi iteration step 5: normres = 0.5364807294115723
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.90e-02, 7.61e-02, 4.18e-01, 3.11e-01, 2.15e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3076366046031801
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.67e-02, 1.79e-01, 2.07e-01, 9.63e-02, 8.54e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10101883010858068
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.34e-03, 1.88e-02, 1.47e-02, 5.53e-02, 7.12e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1061121951218525
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.29e-05